In [1]:
import torch
import torchvision
from torchvision import transforms
from pascal_voc_multi_label_dataset import PascalVOCMultiLabelDataset



In [2]:
# 데이터 전처리 정의
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
# PASCAL VOC 데이터셋 다운로드 및 로드
voc_train = torchvision.datasets.VOCDetection(root='./data', year='2012', image_set='train', download=True, transform=data_transform)
voc_val = torchvision.datasets.VOCDetection(root='./data', year='2012', image_set='val', download=True, transform=data_transform)

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data


In [4]:
train_dataset = PascalVOCMultiLabelDataset(voc_train)
val_dataset = PascalVOCMultiLabelDataset(voc_val)

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)


In [6]:
# !pip install timm
import timm
import torch.nn as nn
import torchvision.models as models

/home/jovyan/.miniconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import torch
import timm
from torch import nn

num_classes = 20  # 변경하려는 클래스 수에 따라 이 값을 수정하세요.

# EfficientNet-B2 모델 생성
model = timm.create_model('efficientnet_b2', pretrained=True)  # 사전 훈련된 가중치를 사용합니다.

# 출력 레이어를 멀티 레이블 작업에 맞게 수정
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, num_classes),
    nn.Sigmoid()
)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b2_ra-bcdf34b7.pth" to /home/jovyan/.cache/torch/hub/checkpoints/efficientnet_b2_ra-bcdf34b7.pth


In [8]:
import torch.optim as optim
# 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [9]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [10]:
# 훈련 및 검증 루프
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
best_val_loss = float('inf')  # 최소 검증 손실 추적을 위한 변수

for epoch in range(num_epochs):
    # 훈련
    model.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    # 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model_efficientnet_b2.pth')

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")


Epoch 1/10, Train Loss: 0.2034, Validation Loss: 0.1063
Epoch 2/10, Train Loss: 0.0912, Validation Loss: 0.0828
Epoch 3/10, Train Loss: 0.0637, Validation Loss: 0.0777
Epoch 4/10, Train Loss: 0.0443, Validation Loss: 0.0803
Epoch 5/10, Train Loss: 0.0313, Validation Loss: 0.0856
Epoch 6/10, Train Loss: 0.0225, Validation Loss: 0.0891
Epoch 7/10, Train Loss: 0.0172, Validation Loss: 0.0912
Epoch 8/10, Train Loss: 0.0141, Validation Loss: 0.0902
Epoch 9/10, Train Loss: 0.0105, Validation Loss: 0.0982
Epoch 10/10, Train Loss: 0.0087, Validation Loss: 0.1019
